# 학교 급식 정보

In [2]:
import neispy
import asyncio
import sys
import pandas as pd
import re

#### schoolFeeding_childId 값 받아온 후 진행

In [3]:
# 년
year = '2021'
# 월
month = '03'
# 년월
montly = year + month
# 학교명
school_name = "늘봄초등학교"

In [4]:
# API 인증키
API_KEY = '2b86f76d17b34031a867e25e85774fbf'

In [5]:
# 필수 인자
# 샘플키로 요청
neis = neispy.Client(KEY = API_KEY)

In [6]:
# 학교이름으로 학교정보를 요청
scinfo = await neis.schoolInfo(SCHUL_NM = school_name)
# 교육청코드
AE = scinfo[0].ATPT_OFCDC_SC_CODE  
# 학교코드
SE = scinfo[0].SD_SCHUL_CODE  

In [7]:
# 학교코드와 교육청 코드 -> 날짜 정보에 해당하는 급식 정보 요청
scmeal = await neis.mealServiceDietInfo(AE, SE, MLSV_YMD = montly)

In [8]:
scmeal_list = []

for scmeal_idx in range(len(scmeal)) :
    schoolFeeding_date = scmeal[scmeal_idx].MLSV_YMD # 급식 일자
    schoolFeeding_type = scmeal[scmeal_idx].MMEAL_SC_NM # 식사 구분명
    schoolFeeding_kcal = scmeal[scmeal_idx].CAL_INFO #칼로리 정보
    
    tuple_dtk = (schoolFeeding_date, 
               schoolFeeding_type, 
               schoolFeeding_kcal)
    
    ntr_list = []
    ntr_info = (scmeal[scmeal_idx].NTR_INFO).split('<br/>') # 영양 정보
    for ntr_idx in range(len(ntr_info)) :
        ntr_temp = ntr_info[ntr_idx].split(' : ')
        ntr_list.append(ntr_temp[1])                                                                                                                                                                                                                                                             
        tuple_ntr = tuple(ntr_list)
    
    schoolFeeding_num = str(scmeal_idx)
    schoolFeeding_id = 'schoolFeeding-' + schoolFeeding_num
    tuple_info = (schoolFeeding_id, '')
    
    
    dish_list = []
    dish_name = (scmeal[scmeal_idx].DDISH_NM).split('<br/>') # 요리 정보
    dish_max_count = 10
    dish_len = len(dish_name)
    for dish_idx in range(dish_len) :
        dish_list.append(re.findall('[가-힣]+', dish_name[dish_idx])[0])
    
        allergy_name = re.findall('\d+', dish_name[dish_idx])
        if len(allergy_name) == 0 : # 알레르기  유발 식재료 번호 X -> 0
            allery_num = '0'
        else : # 알레르기  유발 식재료 번호 O (숫자 ',' 구분)
            allery_num = ''
            for i in range(len(allergy_name)) :
                allery_num = allery_num + allergy_name[i] + ','
            allery_num = allery_num[:-1]
            
        dish_list.append(allery_num)
    
    null_list = []
    for idx in range(dish_len, dish_max_count) : # 빈 열 null로 채우기
        null_list.append('null')
        null_list.append('null')
    
    dish_tuple = tuple(dish_list)
    null_tuple = tuple(null_list)
    
    scmeal_tuple = tuple_info + tuple_dtk + tuple_ntr + dish_tuple + null_tuple
    scmeal_list.append(scmeal_tuple)
    
    print(scmeal_tuple)

('schoolFeeding-000', '', '20210302', '중식', '542.6 Kcal', '85.4', '28.4', '11.0', '193.9', '0.5', '0.4', '13.7', '124.2', '2.8', '기장밥', '0', '쇠고기미역국', '5,16', '돈육고추장불고기', '5,9', '잡채', '2,5,6,10,12,13,16', '배추김치', '9', '미니초코케익', '1,2,5,6', 'null', 'null', 'null', 'null', 'null', 'null', 'null', 'null')
('schoolFeeding-001', '', '20210303', '중식', '580.6 Kcal', '71.1', '28.1', '17.4', '149.3', '0.7', '0.8', '47.2', '116.3', '2.6', '강황굴소스볶음밥', '1,2,5,6,8,13,18', '팽이두부된장국', '15', '방울떡꼬치', '5,6,12', '목살스테이크', '2,5,6,13', '깍두기', '9', '한라봉', '11,12', 'null', 'null', 'null', 'null', 'null', 'null', 'null', 'null')
('schoolFeeding-002', '', '20210304', '중식', '500.1 Kcal', '73.5', '28.8', '11.7', '143.2', '0.3', '0.4', '70.9', '148.1', '5.4', '흑미밥', '0', '순두부찌개', '5,6,9,10,13', '콩나물미나리무침', '5,13', '한우버섯불고기', '5,6,13', '야채애호박전', '1,5,6', '배추김치', '9,13', '딸기', '0', 'null', 'null', 'null', 'null', 'null', 'null')
('schoolFeeding-003', '', '20210305', '중식', '686.2 Kcal', '99.3', '31.0', '21.0', '206.

In [9]:
schoolMeal_df = pd.DataFrame() # 전체 DF

# sol_1
schoolMeal_df[['schoolFeeding_id', 
                  'schoolFeeding_childId', 
                  'schoolFeeding_date', 
                  'schoolFeeding_type', 
                  'schoolFeeding_kcal',
                  'schoolFeeding_carbohydrate', 
                  'schoolFeeding_protein', 
                  'schoolFeeding_fat',
                  'schoolFeeding_vitaminA',
                  'schoolFeeding_thiamine',
                  'schoolFeeding_riboflavin', 
                  'schoolFeeding_vitaminC', 
                  'schoolFeeding_calcium', 
                  'schoolFeeding_iron',
                  'schoolFeedingDetail_foodNo1',
                  'schoolFeedingDetail_foodNo1Allergy',
                  'schoolFeedingDetail_foodNo2',
                  'schoolFeedingDetail_foodNo2Allergy',
                  'schoolFeedingDetail_foodNo3',
                  'schoolFeedingDetail_foodNo3Allergy',
                  'schoolFeedingDetail_foodNo4',
                  'schoolFeedingDetail_foodNo4Allergy',
                  'schoolFeedingDetail_foodNo5',
                  'schoolFeedingDetail_foodNo5Allergy',
                  'schoolFeedingDetail_foodNo6',
                  'schoolFeedingDetail_foodNo6Allergy',
                  'schoolFeedingDetail_foodNo7',
                  'schoolFeedingDetail_foodNo7Allergy',
                  'schoolFeedingDetail_foodNo8',
                  'schoolFeedingDetail_foodNo8Allergy',
                  'schoolFeedingDetail_foodNo9',
                  'schoolFeedingDetail_foodNo9Allergy',
                  'schoolFeedingDetail_foodNo10',
                  'schoolFeedingDetail_foodNo10Allergy']] = pd.DataFrame(scmeal_list)

# sol_2
# schoolMeal_df_df = pd.concat([schoolMeal_df, 
#                               pd.DataFrame(scmeal_list, 
#                                            columns=['schoolFeeding_id', 
#                                                       'schoolFeeding_childId', 
#                                                       'schoolFeeding_date', 
#                                                       'schoolFeeding_type', 
#                                                       'schoolFeeding_kcal',
#                                                       'schoolFeeding_carbohydrate', 
#                                                       'schoolFeeding_protein', 
#                                                       'schoolFeeding_fat',
#                                                       'schoolFeeding_vitaminA',
#                                                       'schoolFeeding_thiamine',
#                                                       'schoolFeeding_riboflavin', 
#                                                       'schoolFeeding_vitaminC', 
#                                                       'schoolFeeding_calcium', 
#                                                       'schoolFeeding_iron',
#                                                       'schoolFeedingDetail_foodNo1',
#                                                       'schoolFeedingDetail_foodNo1Allergy',
#                                                       'schoolFeedingDetail_foodNo2',
#                                                       'schoolFeedingDetail_foodNo2Allergy',
#                                                       'schoolFeedingDetail_foodNo3',
#                                                       'schoolFeedingDetail_foodNo3Allergy',
#                                                       'schoolFeedingDetail_foodNo4',
#                                                       'schoolFeedingDetail_foodNo4Allergy',
#                                                       'schoolFeedingDetail_foodNo5',
#                                                       'schoolFeedingDetail_foodNo5Allergy',
#                                                       'schoolFeedingDetail_foodNo6',
#                                                       'schoolFeedingDetail_foodNo6Allergy',
#                                                       'schoolFeedingDetail_foodNo7',
#                                                       'schoolFeedingDetail_foodNo7Allergy',
#                                                       'schoolFeedingDetail_foodNo8',
#                                                       'schoolFeedingDetail_foodNo8Allergy',
#                                                       'schoolFeedingDetail_foodNo9',
#                                                       'schoolFeedingDetail_foodNo9Allergy',
#                                                       'schoolFeedingDetail_foodNo10',
#                                                       'schoolFeedingDetail_foodNo10Allergy']
#                                           )
#                              ], axis=1)

In [10]:
schoolMeal_df.dtypes

schoolFeeding_id                       object
schoolFeeding_childId                  object
schoolFeeding_date                     object
schoolFeeding_type                     object
schoolFeeding_kcal                     object
schoolFeeding_carbohydrate             object
schoolFeeding_protein                  object
schoolFeeding_fat                      object
schoolFeeding_vitaminA                 object
schoolFeeding_thiamine                 object
schoolFeeding_riboflavin               object
schoolFeeding_vitaminC                 object
schoolFeeding_calcium                  object
schoolFeeding_iron                     object
schoolFeedingDetail_foodNo1            object
schoolFeedingDetail_foodNo1Allergy     object
schoolFeedingDetail_foodNo2            object
schoolFeedingDetail_foodNo2Allergy     object
schoolFeedingDetail_foodNo3            object
schoolFeedingDetail_foodNo3Allergy     object
schoolFeedingDetail_foodNo4            object
schoolFeedingDetail_foodNo4Allergy

In [11]:
schoolMeal_df.head()

,schoolFeeding_id,schoolFeeding_childId,schoolFeeding_date,schoolFeeding_type,schoolFeeding_kcal,schoolFeeding_carbohydrate,schoolFeeding_protein,schoolFeeding_fat,schoolFeeding_vitaminA,schoolFeeding_thiamine,...,schoolFeedingDetail_foodNo6,schoolFeedingDetail_foodNo6Allergy,schoolFeedingDetail_foodNo7,schoolFeedingDetail_foodNo7Allergy,schoolFeedingDetail_foodNo8,schoolFeedingDetail_foodNo8Allergy,schoolFeedingDetail_foodNo9,schoolFeedingDetail_foodNo9Allergy,schoolFeedingDetail_foodNo10,schoolFeedingDetail_foodNo10Allergy
0,schoolFeeding-000,,20210302,중식,542.6 Kcal,85.4,28.4,11.0,193.9,0.5,...,미니초코케익,"1,2,5,6",null,null,null,null,null,null,null,null
1,schoolFeeding-001,,20210303,중식,580.6 Kcal,71.1,28.1,17.4,149.3,0.7,...,한라봉,"11,12",null,null,null,null,null,null,null,null
2,schoolFeeding-002,,20210304,중식,500.1 Kcal,73.5,28.8,11.7,143.2,0.3,...,배추김치,"9,13",딸기,0,null,null,null,null,null,null
3,schoolFeeding-003,,20210305,중식,686.2 Kcal,99.3,31.0,21.0,206.0,0.4,...,케이준샐러드,"13,1,2,5,6,12",null,null,null,null,null,null,null,null
4,schoolFeeding-004,,20210308,중식,660.6 Kcal,94.8,30.7,19.0,161.6,0.2,...,사과,0,null,null,null,null,null,null,null,null


In [12]:
schoolFeeding_df = schoolMeal_df.loc[:, ['schoolFeeding_id', 
                                         'schoolFeeding_childId',
                                         'schoolFeeding_date',
                                         'schoolFeeding_type',
                                         'schoolFeeding_kcal',
                                         'schoolFeeding_carbohydrate',
                                         'schoolFeeding_protein',
                                         'schoolFeeding_fat',
                                         'schoolFeeding_vitaminA',
                                         'schoolFeeding_thiamine',
                                         'schoolFeeding_riboflavin',
                                         'schoolFeeding_vitaminC',
                                         'schoolFeeding_calcium',
                                         'schoolFeeding_iron']]
schoolFeeding_df.dtypes

schoolFeeding_id              object
schoolFeeding_childId         object
schoolFeeding_date            object
schoolFeeding_type            object
schoolFeeding_kcal            object
schoolFeeding_carbohydrate    object
schoolFeeding_protein         object
schoolFeeding_fat             object
schoolFeeding_vitaminA        object
schoolFeeding_thiamine        object
schoolFeeding_riboflavin      object
schoolFeeding_vitaminC        object
schoolFeeding_calcium         object
schoolFeeding_iron            object
dtype: object

In [13]:
schoolFeedingDetail_df = schoolMeal_df.loc[:, ['schoolFeeding_id', 
                                                  'schoolFeedingDetail_foodNo1',
                                                  'schoolFeedingDetail_foodNo1Allergy',
                                                  'schoolFeedingDetail_foodNo2',
                                                  'schoolFeedingDetail_foodNo2Allergy',
                                                  'schoolFeedingDetail_foodNo3',
                                                  'schoolFeedingDetail_foodNo3Allergy',
                                                  'schoolFeedingDetail_foodNo4',
                                                  'schoolFeedingDetail_foodNo4Allergy',
                                                  'schoolFeedingDetail_foodNo5',
                                                  'schoolFeedingDetail_foodNo5Allergy',
                                                  'schoolFeedingDetail_foodNo6',
                                                  'schoolFeedingDetail_foodNo6Allergy',
                                                  'schoolFeedingDetail_foodNo7',
                                                  'schoolFeedingDetail_foodNo7Allergy',
                                                  'schoolFeedingDetail_foodNo8',
                                                  'schoolFeedingDetail_foodNo8Allergy',
                                                  'schoolFeedingDetail_foodNo9',
                                                  'schoolFeedingDetail_foodNo9Allergy',
                                                  'schoolFeedingDetail_foodNo10',
                                                  'schoolFeedingDetail_foodNo10Allergy']]
schoolFeedingDetail_df.dtypes

schoolFeeding_id                       object
schoolFeedingDetail_foodNo1            object
schoolFeedingDetail_foodNo1Allergy     object
schoolFeedingDetail_foodNo2            object
schoolFeedingDetail_foodNo2Allergy     object
schoolFeedingDetail_foodNo3            object
schoolFeedingDetail_foodNo3Allergy     object
schoolFeedingDetail_foodNo4            object
schoolFeedingDetail_foodNo4Allergy     object
schoolFeedingDetail_foodNo5            object
schoolFeedingDetail_foodNo5Allergy     object
schoolFeedingDetail_foodNo6            object
schoolFeedingDetail_foodNo6Allergy     object
schoolFeedingDetail_foodNo7            object
schoolFeedingDetail_foodNo7Allergy     object
schoolFeedingDetail_foodNo8            object
schoolFeedingDetail_foodNo8Allergy     object
schoolFeedingDetail_foodNo9            object
schoolFeedingDetail_foodNo9Allergy     object
schoolFeedingDetail_foodNo10           object
schoolFeedingDetail_foodNo10Allergy    object
dtype: object

In [14]:
schoolFeedingDetail_df.rename(columns={'schoolFeeding_id':'schoolFeedingDetail_schoolFeedingId'},
                              inplace=True)

## **_schoolFeeding DF_** 

In [15]:
schoolFeeding_df.head()

,schoolFeeding_id,schoolFeeding_childId,schoolFeeding_date,schoolFeeding_type,schoolFeeding_kcal,schoolFeeding_carbohydrate,schoolFeeding_protein,schoolFeeding_fat,schoolFeeding_vitaminA,schoolFeeding_thiamine,schoolFeeding_riboflavin,schoolFeeding_vitaminC,schoolFeeding_calcium,schoolFeeding_iron
0,schoolFeeding-000,,20210302,중식,542.6 Kcal,85.4,28.4,11.0,193.9,0.5,0.4,13.7,124.2,2.8
1,schoolFeeding-001,,20210303,중식,580.6 Kcal,71.1,28.1,17.4,149.3,0.7,0.8,47.2,116.3,2.6
2,schoolFeeding-002,,20210304,중식,500.1 Kcal,73.5,28.8,11.7,143.2,0.3,0.4,70.9,148.1,5.4
3,schoolFeeding-003,,20210305,중식,686.2 Kcal,99.3,31.0,21.0,206.0,0.4,0.4,17.0,168.4,8.4
4,schoolFeeding-004,,20210308,중식,660.6 Kcal,94.8,30.7,19.0,161.6,0.2,0.6,22.2,189.1,5.8


## **_schoolFeedingDetail DF_** 

In [39]:
schoolFeedingDetail_df.head()

,schoolFeedingDetail_schoolFeedingId,schoolFeedingDetail_foodNo1,schoolFeedingDetail_foodNo1Allergy,schoolFeedingDetail_foodNo2,schoolFeedingDetail_foodNo2Allergy,schoolFeedingDetail_foodNo3,schoolFeedingDetail_foodNo3Allergy,schoolFeedingDetail_foodNo4,schoolFeedingDetail_foodNo4Allergy,schoolFeedingDetail_foodNo5,...,schoolFeedingDetail_foodNo6,schoolFeedingDetail_foodNo6Allergy,schoolFeedingDetail_foodNo7,schoolFeedingDetail_foodNo7Allergy,schoolFeedingDetail_foodNo8,schoolFeedingDetail_foodNo8Allergy,schoolFeedingDetail_foodNo9,schoolFeedingDetail_foodNo9Allergy,schoolFeedingDetail_foodNo10,schoolFeedingDetail_foodNo10Allergy
0,schoolFeeding-000,기장밥,0,쇠고기미역국,"5,16",돈육고추장불고기,"5,9",잡채,"2,5,6,10,12,13,16",배추김치,...,미니초코케익,"1,2,5,6",null,null,null,null,null,null,null,null
1,schoolFeeding-001,강황굴소스볶음밥,"1,2,5,6,8,13,18",팽이두부된장국,15,방울떡꼬치,"5,6,12",목살스테이크,"2,5,6,13",깍두기,...,한라봉,"11,12",null,null,null,null,null,null,null,null
2,schoolFeeding-002,흑미밥,0,순두부찌개,"5,6,9,10,13",콩나물미나리무침,"5,13",한우버섯불고기,"5,6,13",야채애호박전,...,배추김치,"9,13",딸기,0,null,null,null,null,null,null
3,schoolFeeding-003,차수수밥,0,참치김치찌개,"9,13",마파두부,"5,6,10,12,13",총각김치,13,수제마늘토스트,...,케이준샐러드,"13,1,2,5,6,12",null,null,null,null,null,null,null,null
4,schoolFeeding-004,차조밥,0,오징어무국,"5,6,13",깐쇼새우떡조림,"1,5,6,9,12,13",한우메추리알장조림,"1,5,6,10,13,16",봄동달래겉절이,...,사과,0,null,null,null,null,null,null,null,null


# 학사일정

In [35]:
# 학교코드와 교육청 코드로 2019년 3월 7일날 학사일정 요청
scschedule = await neis.SchoolSchedule(AE, SE, AA_YMD=202004)
schedule = scschedule[0].EVENT_NM  # 학사일정명 가져옴

In [38]:
for schedule_idx in range(0, len(scschedule)) :
    day = scschedule[schedule_idx].AA_YMDx
    schedule = scschedule[schedule_idx].EVENT_NM 
    print(day + ", " + schedule)

20200401, 코로나-19 휴업일
20200402, 코로나-19 휴업일
20200403, 코로나-19 휴업일
20200404, 토요휴업일
20200406, 코로나-19 휴업일
20200407, 코로나-19 휴업일
20200408, 코로나-19 휴업일
20200409, 코로나-19 휴업일
20200410, 코로나-19 휴업일
20200411, 토요휴업일
20200413, 코로나-19 휴업일
20200414, 코로나-19 휴업일
20200415, 국회의원선거
20200416, 시업식
20200416, 코로나-19 휴업일
20200417, 코로나-19 휴업일
20200418, 토요휴업일
20200420, 시업식
20200420, 입학식
20200425, 토요휴업일
20200430, 부처님오신날(석가탄신일)
